In [17]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

## 9.1 在函数上添加包装器

如果你想使用额外的代码包装一个函数,可以定义一个装饰器函数,例如:

<span class="girk">`@wraps(func)` 注解是很重要的,它能保留原始函数的元数据</span>

In [7]:
from functools import wraps
import time

def timethis(func):
    @wraps(func) # ? 有无这行都可以，那么这行有什么用？
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

@timethis
def countdown(n):
    while n > 0:
        n -= 1

countdown(10000)

countdown 0.0008330345153808594


一个装饰器就是一个函数,它接受一个函数作为参数并返回一个新的函数。

内置的装饰器比如 @staticmethod, @classmethod,@property 原理也是一样的。  
以classmethod为例：

In [8]:
class A:
    @classmethod
    def method(cls):
        pass

# B类实现与类A等价
class B:
    def method(cls):
        pass
    method = classmethod(method)

## 9.2 创建装饰器时保留函数元信息

如果上述的`@wraps(func)`, 这个函数的重要的元信息比如名字、文档字符串、注解和参数签名都丢失了。

### 试试不加`@wraps`的效果

In [42]:
import time
from functools import wraps
def timethis(func):
    '''Decorator that reports the execution time.'''
#     @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

@timethis
def countdown(n:int):
    """count down doc"""
    while n > 0:
        n -= 1

# 打印countdown函数的元信息，
# 期待值是
# 'countdown'
# 'count down doc'
# 缺少@wraps打印丢失了元信息数据
countdown.__name__
countdown.__doc__
countdown.__annotations__

'wrapper'

{}

### 使用@wraps，保留被包装的函数的元数据

In [44]:
import time
from functools import wraps
def timethis(func):
    '''Decorator that reports the execution time.'''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

# 使用这个被包装后的函数并检查它的元信息:

@timethis
def countdown(n:int):
    """count down doc"""
    while n > 0:
        n -= 1

countdown(100000)

# 打印countdown函数的元信息
countdown.__name__
countdown.__doc__
countdown.__annotations__

countdown 0.009287118911743164


'countdown'

'count down doc'

{'n': int}

@wraps 有一个重要特征是它能让你通过属性 __wrapped__ 直接访问被包装函数。
例如:

In [45]:
countdown.__wrapped__(100000) #只运行了，没额外操作

__wrapped__ 属性还能让被装饰函数正确暴露底层的参数签名信息。

In [46]:
from inspect import signature
print(signature(countdown))

(n: int)


## 9.3 解除一个装饰器

假设装 饰 器 是 通 过 @wraps (参参 考__wrapped__ 属性来访问原始函数:9.2 小 节) 来 实 现 的, 那 么 你 可__wrapped__ 属性来访问原始函数:

直接访问未包装的原始函数在调试、内省和其他函数操作时是很有用的。但局限在于:  
- 仅仅适用于在包装器中正确使用了 @wraps 或者直接设置了 __wrapped__属性的情况。
- 如果有多个包装器,那么访问 __wrapped__ 属性的行为是不可预知的,应该避免这样做。

In [50]:
from functools import wraps

def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper

@decorator1
@decorator2
def add(x, y):
    return x + y

In [51]:
add(2, 3)

Decorator 1
Decorator 2


5

In [53]:
add.__wrapped__(2, 3)

Decorator 2


5

## 9.4 定义一个带参数的装饰器

In [56]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    """
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper

    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, "example")
def spam():
    print('Spam!')

add(1,2)
print('---------')
spam()

3

spam


---------
Spam!


#### 原理: